# Testing score aggregation for languages with no white spaces

Problem: The explainability method produces one score per token. The scores need to be aligned with words. In languages with white space, this can be done by considering sequences divided by spaces as individual words. However, in languages where white space is not used or it has a different meaning (i.e. sentence break), a parser needs to be used:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg" style="undefined;table-layout: fixed; width: 460px">
<colgroup>
<col style="width: 157.2px">
<col style="width: 43.2px">
<col style="width: 45.2px">
<col style="width: 59.2px">
<col style="width: 42.2px">
<col style="width: 51.2px">
<col style="width: 62.2px">
</colgroup>
<thead>
  <tr>
    <th class="tg-0lax"></th>
    <th class="tg-baqh" colspan="6">tokens</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">model tokenization</td>
    <td class="tg-0pky">我</td>
    <td class="tg-0pky" colspan="2">的名字</td>
    <td class="tg-0pky">是</td>
    <td class="tg-0pky">明</td>
    <td class="tg-0pky">珠</td>
  </tr>
  <tr>
    <td class="tg-0pky">model scores</td>
    <td class="tg-0pky">0.2</td>
    <td class="tg-0pky" colspan="2">0.14</td>
    <td class="tg-0pky">-0.1</td>
    <td class="tg-0pky">-0.05</td>
    <td class="tg-0pky">0.03</td>
  </tr>
  <tr>
    <td class="tg-0pky">"correct" tokenization</td>
    <td class="tg-0pky">我</td>
    <td class="tg-0pky">的</td>
    <td class="tg-0pky">名字</td>
    <td class="tg-0pky">是</td>
    <td class="tg-0pky" colspan="2">明珠</td>
  </tr>
  <tr>
    <td class="tg-0pky">aggregated scores</td>
    <td class="tg-0pky">0.2</td>
    <td class="tg-0pky">0.14</td>
    <td class="tg-0pky">0.14</td>
    <td class="tg-0pky">-0.1</td>
    <td class="tg-0pky" colspan="2">-0.01</td>
  </tr>
</tbody>
</table>


In this file the alignment is tested using 4 languages supported by spaCy which use white space as sentence divider and not between words: Simplified Chinese, Japanese, Korean, and Thai.

In [1]:
import spacy
!python -m spacy download zh_core_web_md
!python -m spacy download ja_core_news_md
!python -m spacy download ko_core_news_sm
!pip install spacy_thai

2024-01-15 08:30:05.707942: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 08:30:05.708054: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 08:30:05.710277: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 08:30:08.332566: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 MB 7.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_md')
2024-01-15 08:30:27.436609: E external/local_xla/xla/stream_


## Chinese

### Example usage of spaCy

In [2]:
nlp = spacy.load("zh_core_web_md")

In [3]:
from spacy.lang.zh.examples import sentences as zh_sentences

doc = nlp(zh_sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

作为语言而言，为世界使用人数最多的语言，目前世界有五分之一人口做为母语。
作为 ADP case
语言 NOUN nmod:prep
而言 PART case
， PUNCT punct
为 ADP case
世界 NOUN compound:nn
使用 NOUN compound:nn
人数 NOUN nsubj
最多 VERB acl
的 PART mark
语言 NOUN ROOT
， PUNCT punct
目前 NOUN nmod:tmod
世界 NOUN dep
有 VERB conj
五分之一 NUM dep
人口 NOUN nsubj
做为 VERB ccomp
母语 NOUN dobj
。 PUNCT punct


### Example of tokenisation

In [4]:
from transformers import AutoTokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
text = "作为语言而言，为世界使用人数最多的语言，目前世界有五分之一人口做为母语。"
for i in tokenizer(text).input_ids:
    print(tokenizer.decode(i))

<s>

作为
语言
而言
,
为
世界
使用
人数
最多的
语言
,
目前
世界
有
五
分
之一
人口
做
为
母
语
。
</s>


### Adding scores (and transforming to the format explain.py uses)

In [7]:
import numpy as np
import itertools
parsed_text = [i.text for i in nlp(zh_sentences[0])]
tokenized_text = [tokenizer.decode(i) for i in tokenizer(zh_sentences[0]).input_ids]
scores = [0.2+np.round(np.sin(0.8*i/np.pi), decimals=1) for i in range(len(tokenized_text))]

### Comparison:

In [8]:
for i,j,k in itertools.zip_longest(scores, tokenized_text, parsed_text):
    print(f'{i} \t {j} \t {k}')

0.2 	 <s> 	 作为
0.5 	  	 语言
0.7 	 作为 	 而言
0.8999999999999999 	 语言 	 ，
1.1 	 而言 	 为
1.2 	 , 	 世界
1.2 	 为 	 使用
1.2 	 世界 	 人数
1.1 	 使用 	 最多
1.0 	 人数 	 的
0.8 	 最多的 	 语言
0.5 	 语言 	 ，
0.30000000000000004 	 , 	 目前
0.0 	 目前 	 世界
-0.2 	 世界 	 有
-0.39999999999999997 	 有 	 五分之一
-0.6000000000000001 	 五 	 人口
-0.7 	 分 	 做为
-0.8 	 之一 	 母语
-0.8 	 人口 	 。
-0.7 	 做 	 None
-0.6000000000000001 	 为 	 None
-0.39999999999999997 	 母 	 None
-0.2 	 语 	 None
0.0 	 。 	 None
0.30000000000000004 	 </s> 	 None


## Alignment function

In [9]:
import re
special_tokens = ["<s>", "</s>"]

ASSERTION_MSG = lambda p,t,sp: '''
    Given parsing of the sentence cannot be aligned.
    Checking these might help:
    - if string.isalpha() is applicable for your language
    - the variable "special_tokens" contains the special tokens of your model,
    - the two tokenizations are of the same sentence.
    ''' + f'\nThe sentences are \n {p} \n {t} \nand the special tokens are {sp}.'

def align(parsed, tokenized, scores):
    """
    Function to align two different tokenizations of the same text.
    E.g. for Chinese, the model tokenizer might tokenize
    母语 (mother tongue) as "母" and "语".
    -> if you have defined a better tokenized, give both tokenizations
    to this function and it will align them and the scores
    associated with the model tokenizer.
    All punctuation is removed for this tokenization.
    -> would be removed in the keyword extraction step anyway.
    """

    parsed = [re.sub(r'[^\D\s]|[^\w\s]', '', p) for p in parsed if any(j.isalpha() for j in p)]
    to_be_dropped = [i for i in range(len(tokenized)) if any(j.isalpha() for j in tokenized[i]) and tokenized[i] not in special_tokens]
    tokenized = [re.sub(r'[^\D\s]|[^\w\s]', '',tokenized[i]) for i in to_be_dropped]
    scores = [scores[i] for i in to_be_dropped]


    assert "".join(parsed)=="".join(tokenized), ASSERTION_MSG(parsed, tokenized, special_tokens)

    # align
    # t_ind contains the current index we're at now in the tokenized (by model) sentence
    t_ind=0
    agg_scores = np.zeros(len(parsed))
    # for each "real tokenization" (here "parsed")
    for p_ind in range(len(parsed)):
        sub_scores = []
        for p in parsed[p_ind]:     # for each char in that tokenization
            if p == tokenized[t_ind][0]:        # if theres a match
                sub_scores.append(scores[t_ind])    # add the score for the character
                if len(tokenized[t_ind])>1:         # remove the character for next calculation
                    tokenized[t_ind] = tokenized[t_ind][1:]
                else:
                    tokenized[t_ind]="-"        # not necessary but helped debug
                    t_ind+=1                    # move to next index
            else:
                raise Exception("Alignment impossible for unforeseen reasons.")       # despite assertion, something went wrong
        agg_scores[p_ind] = np.mean(sub_scores)     # aggregate the scores TODO: method

    return parsed, agg_scores.tolist()

align(parsed_text, tokenized_text, scores)


(['作为',
  '语言',
  '而言',
  '为',
  '世界',
  '使用',
  '人数',
  '最多',
  '的',
  '语言',
  '目前',
  '世界',
  '有',
  '五分之一',
  '人口',
  '做为',
  '母语'],
 [0.7,
  0.8999999999999999,
  1.1,
  1.2,
  1.2,
  1.1,
  1.0,
  0.8,
  0.8,
  0.5,
  0.0,
  -0.2,
  -0.39999999999999997,
  -0.7250000000000001,
  -0.8,
  -0.65,
  -0.3])

## Japanese

In [10]:
from spacy.lang.ja.examples import sentences as ja_sentences

nlp = spacy.load("ja_core_news_md")
doc = nlp(ja_sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

アップルがイギリスの新興企業を１０億ドルで購入を検討
アップル PROPN nsubj
が ADP case
イギリス PROPN nmod
の ADP case
新興 NOUN compound
企業 NOUN obj
を ADP case
１０億 NUM punct
ドル NOUN obl
で ADP case
購入 NOUN obj
を ADP case
検討 NOUN ROOT


In [11]:
parsed_text = [i.text for i in nlp(ja_sentences[0])]
tokenized_text = [tokenizer.decode(i) for i in tokenizer(ja_sentences[0]).input_ids]
scores = [0.2+np.round(np.sin(0.8*i/np.pi), decimals=1) for i in range(len(tokenized_text))]

In [12]:
for i,j,k in itertools.zip_longest(scores, tokenized_text, parsed_text):
    print(f'{i} \t {j} \t {k}')

0.2 	 <s> 	 アップル
0.5 	  	 が
0.7 	 アップ 	 イギリス
0.8999999999999999 	 ル 	 の
1.1 	 が 	 新興
1.2 	 イギリス 	 企業
1.2 	 の 	 を
1.2 	 新興 	 １０億
1.1 	 企業 	 ドル
1.0 	 を 	 で
0.8 	 10 	 購入
0.5 	 億 	 を
0.30000000000000004 	 ドル 	 検討
0.0 	 で購入 	 None
-0.2 	 を検討 	 None
-0.39999999999999997 	 </s> 	 None


In [13]:
align(parsed_text, tokenized_text, scores)

(['アップル', 'が', 'イギリス', 'の', '新興', '企業', 'を', '億', 'ドル', 'で', '購入', 'を', '検討'],
 [0.7499999999999999,
  1.1,
  1.2,
  1.2,
  1.2,
  1.1,
  1.0,
  0.5,
  0.30000000000000004,
  0.0,
  0.0,
  -0.2,
  -0.2])

## Testing with more examples

I'm testing with Chinese, Japanese, Korean, and Thai.

In [14]:
nlp = spacy.load("zh_core_web_md")
for doc in zh_sentences:
    parsed_text = [i.text for i in nlp(doc)]
    tokenized_text = [tokenizer.decode(i) for i in tokenizer(doc).input_ids]
    scores = [0.2+np.round(np.sin(0.8*i/np.pi), decimals=1) for i in range(len(tokenized_text))]
    print(align(parsed_text, tokenized_text, scores))

(['作为', '语言', '而言', '为', '世界', '使用', '人数', '最多', '的', '语言', '目前', '世界', '有', '五分之一', '人口', '做为', '母语'], [0.7, 0.8999999999999999, 1.1, 1.2, 1.2, 1.1, 1.0, 0.8, 0.8, 0.5, 0.0, -0.2, -0.39999999999999997, -0.7250000000000001, -0.8, -0.65, -0.3])
(['汉语', '有', '多种', '分支', '当中', '官话', '最为', '流行', '为', '中华', '人民', '共和国', '的', '国家', '通用', '语言', '又', '称为', '普通话', '以及', '中华', '民国', '的', '国语'], [0.7, 0.8999999999999999, 1.1, 1.2, 1.1, 0.9, 0.5, 0.30000000000000004, -0.2, -0.39999999999999997, -0.39999999999999997, -0.39999999999999997, -0.6000000000000001, -0.6000000000000001, -0.7, -0.8, -0.7, -0.6000000000000001, -0.3333333333333333, 0.5, 0.8, 1.0, 1.2, 1.2])
(['此外', '中文', '还是', '联合国', '正式', '语文', '并', '被', '上海', '合作', '组织', '等', '国际', '组织', '采用', '为', '官方', '语言'], [0.5, 0.8999999999999999, 1.1, 1.2, 1.2, 1.15, 0.8, 0.5, 0.30000000000000004, 0.0, -0.2, -0.39999999999999997, -0.6000000000000001, -0.7, -0.8, -0.8, -0.7, -0.6000000000000001])
(['在', '中国', '大陆', '汉语', '通称', '为', '汉语'], [0.7, 0.7, 

In [15]:
nlp = spacy.load("ja_core_news_md")
for doc in ja_sentences:
    parsed_text = [i.text for i in nlp(doc)]
    tokenized_text = [tokenizer.decode(i) for i in tokenizer(doc).input_ids]
    scores = [0.2+np.round(np.sin(0.8*i/np.pi), decimals=1) for i in range(len(tokenized_text))]
    print(align(parsed_text, tokenized_text, scores))

(['アップル', 'が', 'イギリス', 'の', '新興', '企業', 'を', '億', 'ドル', 'で', '購入', 'を', '検討'], [0.7499999999999999, 1.1, 1.2, 1.2, 1.2, 1.1, 1.0, 0.5, 0.30000000000000004, 0.0, 0.0, -0.2, -0.2])
(['自動', '運転', '車', 'の', '損害', '賠償', '責任', '自動車', 'メーカー', 'に', '一定', 'の', '負担', 'を', '求める'], [0.7, 0.8999999999999999, 1.1, 1.2, 1.2, 1.2, 1.1, 0.8000000000000002, 0.5, 0.30000000000000004, 0.0, -0.2, -0.39999999999999997, -0.6000000000000001, -0.6000000000000001])
(['歩道', 'を', '走る', '自動', '配達', 'ロボ', 'サンフランシスコ', '市', 'が', '走行', '禁止', 'を', '検討'], [0.7999999999999999, 1.1, 1.2, 1.2, 1.05, 0.65, -0.3875, -0.8, -0.7, -0.5, -0.2, 0.0, 0.0])
(['ロンドン', 'は', 'イギリス', 'の', '大', '都市', 'です'], [0.85, 1.2, 1.2, 1.2, 1.1, 1.0, 0.8])


In [16]:
from spacy.lang.ko.examples import sentences as ko_sentences
nlp = spacy.load("ko_core_news_sm")
for doc in ko_sentences:
    parsed_text = [i.text for i in nlp(doc)]
    tokenized_text = [tokenizer.decode(i) for i in tokenizer(doc).input_ids]
    scores = [0.2+np.round(np.sin(0.8*i/np.pi), decimals=1) for i in range(len(tokenized_text))]
    print(align(parsed_text, tokenized_text, scores))

(['애플이', '영국의', '스타트업을', '억', '달러에', '인수하는', '것을', '알아보고', '있다'], [0.5666666666666667, 0.9666666666666667, 1.2, 1.1, 0.7000000000000001, -0.024999999999999994, -0.39999999999999997, -0.625, -0.8])
(['자율주행', '자동차의', '손해', '배상', '책임이', '제조', '업체로', '옮겨', '가다'], [0.7999999999999999, 1.2, 1.15, 0.9, 0.43333333333333335, 0.0, -0.26666666666666666, -0.75, -0.75])
(['샌프란시스코', '시가', '자동', '배달', '로봇의', '보도', '주행', '금지를', '검토', '중이라고', '합니다'], [1.05, 1.15, 1.0, 0.65, 0.20000000000000004, -0.2, -0.5, -0.7333333333333334, -0.8, -0.625, -0.39999999999999997])
(['런던은', '영국의', '수도이자', '가장', '큰', '도시입니다'], [0.9, 1.2, 1.15, 1.0, 0.8, 0.38])


In [17]:
th_sentences = ["ช่วยพูดช้าๆได้ไหม", "ช่วยอธิบายให้ฉันฟังหน่อย", "ช่วยยกตัวอย่างให้หน่อย"]
import spacy_thai
nlp=spacy_thai.load()
for doc in th_sentences:
    parsed_text = [i.text for i in nlp(doc)]
    tokenized_text = [tokenizer.decode(i) for i in tokenizer(doc).input_ids]
    scores = [0.2+np.round(np.sin(0.8*i/np.pi), decimals=1) for i in range(len(tokenized_text))]
    print(align(parsed_text, tokenized_text, scores))

(['ชวย', 'พด', 'ชาๆ', 'ได', 'ไหม'], [0.5, 0.7, 0.9666666666666667, 1.2, 1.2])
(['ชวย', 'อธบาย', 'ให', 'ฉน', 'ฟง', 'หนอย'], [0.5, 0.7, 0.8999999999999999, 1.1, 1.2, 1.2])
(['ชวย', 'ยกตวอยาง', 'ให', 'หนอย'], [0.5, 0.8499999999999999, 1.1, 1.2])
